In [2]:
import numpy as np
import pandas as pd

In [3]:
train_data = pd.read_csv('Train_Data.csv')
test_data = pd.read_csv('Test_Data.csv')
train_data.head()

,date,campaign,adgroup,ad,impressions,clicks,cost,conversions,revenue
0,01-08-2020,campaign 1,adgroup 1,ad 1,24,6,0.08,0,0.00
1,01-08-2020,campaign 1,adgroup 2,ad 1,1,0,0.00,0,0.00
2,01-08-2020,campaign 1,adgroup 3,ad 1,13,4,0.04,0,0.00
3,01-08-2020,campaign 1,adgroup 4,ad 1,5,4,0.08,0,0.00
4,01-08-2020,campaign 1,adgroup 1,ad 2,247,126,1.29,4,925.71


In [4]:
train_data.drop('date', axis=1, inplace=True)
train_data.drop('campaign', axis=1, inplace=True)
test_data.drop('date', axis=1, inplace=True)
test_data.drop('campaign', axis=1, inplace=True)
train_data.drop('ad', axis=1, inplace=True)
test_data.drop('ad', axis=1, inplace=True)

In [5]:
train_data = pd.get_dummies(train_data)
test_data = pd.get_dummies(test_data)
train_data.head()

,impressions,clicks,cost,conversions,revenue,adgroup_adgroup 1,adgroup_adgroup 2,adgroup_adgroup 3,adgroup_adgroup 4
0,24,6,0.08,0,0.00,1,0,0,0
1,1,0,0.00,0,0.00,0,1,0,0
2,13,4,0.04,0,0.00,0,0,1,0
3,5,4,0.08,0,0.00,0,0,0,1
4,247,126,1.29,4,925.71,1,0,0,0


In [8]:
X_train = train_data.drop(['revenue'], axis='columns')
y_train = train_data['revenue']
X_test = test_data

In [6]:
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

In [7]:
def get_models():
	models = dict()
	models['knn'] = KNeighborsRegressor()
	models['cart'] = DecisionTreeRegressor()
	models['svm'] = SVR()
	return models

In [18]:
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std


def evaluate_model(model, X_train, y_train):
	cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
	scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1, error_score='raise')
	return scores

In [19]:
models = get_models()

In [20]:
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, X_train, y_train)
	results.append(scores)
	names.append(name)
	print('>%s %.3f (%.3f)' % (name, mean(scores), std(scores)))

>knn -142.665 (25.266)
>cart -93.223 (13.926)
>svm -199.099 (46.512)


In [21]:
cart = DecisionTreeRegressor()
cart.fit(X_train, y_train)

DecisionTreeRegressor()

In [22]:
preds =  cart.predict(X_test)

In [23]:
preds = preds.astype('int64')
preds

array([ 174,    0,    0,    0,    0,    0,  174,    0,  174,    0, 1328,
          0,    0,  250,    0,  174,    0,  174,    0,  174,    0,    0,
          0, 1328,    0,    0,  947,  174,    0,    0,  718,    0, 1368,
        537,   21,    0,  174,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,  250,  174,
         76,    0,    0,    0,  740,    0,  250,    0,  250,    0,    0,
       1328,  174,    0,    0,  250,    0,    0,    0,  174,  174,  947,
          0,    0,    0, 1328,   76,    0,  174,    0,    0,  947, 1598,
          0,  250,  283,  254,    0, 1619,  174,    0,    0,    0,    0,
          0,  174,    0,    0,  174,    0, 3206,    0,  283,    0,  174,
          0,  947,    0,  537,    0, 1904,    0, 3206, 1328,  174,    0,
          0, 1328,    0,    0,    0, 1328,    0,  174,  174,    0,    0,
          0,    0,  947,    0,    0,    0, 1328,    0,  174,    0,    0,
          0,   21,    0,    0,  847, 1164,    0,   

In [24]:
prediction = pd.DataFrame(preds, columns=['revenue']).to_csv('dectree.csv', index=False)